In [1]:
from copy import deepcopy

import torch.optim as optim
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from tqdm import tqdm
import wandb

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim 
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

In [3]:
from senmodel.model.utils import *
from senmodel.metrics.nonlinearity_metrics import *
from senmodel.metrics.edge_finder import *
from senmodel.metrics.train_metrics import *
from senmodel.train.train import *

In [4]:
torch.manual_seed(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
housing = fetch_california_housing()
X, y = housing.data, housing.target

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32))
val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32))

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [6]:
hyperparams = {
    "num_epochs": 128,
    "metric": MagnitudeL2Metric(nn.CrossEntropyLoss()),
    "aggregation_mode": "mean",
    "choose_thresholds": {"fc0": 0.01, "fc1": 0.01},
    "replace_layers": ["fc0", "fc1"],
    "threshold": 0.004,
    "min_delta_epoch_replace": 8,
    "window_size": 16,
    "lr": 4e-5,
    "delete_after": 2,    
    "task_type": 'regression'
}

name = ", ".join(
    f"{key}: {value.__class__.__name__ if key == 'metric' else value}"
    for key, value in hyperparams.items()
)

name

"num_epochs: 128, metric: MagnitudeL2Metric, aggregation_mode: mean, choose_thresholds: {'fc0': 0.1, 'fc1': 0.1}, replace_layers: ['fc0', 'fc1'], threshold: 0.004, min_delta_epoch_replace: 8, window_size: 16, lr: 4e-05, delete_after: 2, task_type: regression"

In [7]:
# Define the model
class SimpleFCN(nn.Module):
    def __init__(self, input_size=8):
        super(SimpleFCN, self).__init__()
        self.fc0 = nn.Linear(input_size, 16)
        self.fc1 = nn.Linear(16, 1)
        self.act = nn.ReLU()

    def forward(self, x):
        x = self.fc1(self.act(self.fc0(x)))
        return x

In [8]:
model = SimpleFCN()
sparse_model = convert_dense_to_sparse_network(model, layers=[model.fc0, model.fc1])

In [9]:
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: vanyamironov to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [10]:
wandb.finish()
run = wandb.init(
    project="self-expanding-nets-california",
    name=f"trash",
    # name=f"to trash",
)

In [11]:
import warnings

warnings.filterwarnings("ignore")


In [12]:
criterion = nn.MSELoss()
train_sparse_recursive(sparse_model, train_loader, train_loader, val_loader, criterion, hyperparams)

  0%|          | 0/258 [00:00<?, ?it/s]

100%|██████████| 258/258 [00:00<00:00, 728.28it/s]


Epoch 1/128, Train Loss: 6.7277, Val Loss: 6.4689, Val Accuracy: 6.3498


100%|██████████| 258/258 [00:00<00:00, 1276.76it/s]


Epoch 2/128, Train Loss: 6.3934, Val Loss: 6.1470, Val Accuracy: 6.0264


...

Epoch 126/128, Train Loss: 1.3382, Val Loss: 1.3170, Val Accuracy: 1.2820


100%|██████████| 258/258 [00:01<00:00, 253.49it/s]


Epoch 127/128, Train Loss: 1.3397, Val Loss: 1.3160, Val Accuracy: 1.2971


100%|██████████| 258/258 [00:01<00:00, 219.85it/s]


Epoch 128/128, Train Loss: 1.3392, Val Loss: 1.3149, Val Accuracy: 1.2860
